In [31]:
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
import xarray as xr


In [2]:
#Replace with file path for this directory on your machine
!pip install -e "C:/Users/shubg/Desktop/GitHub/assetraMP"

Obtaining file:///C:/Users/shubg/Desktop/GitHub/assetraMP
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for assetra (pyproject.toml): started
  Building editable for assetra (pyproject.toml): finished with status 'done'
  Created wheel for assetra: filename=assetra-1.0.3-py3-none-any.whl size=2693 sha256=e87d9d08060b707e08cfdf8292114cb056f8de4eb809927c12bb975a75c84afb
  Stored in directory: C:\Users\shubg\AppData\Local\Temp\pip-ephem-wheel-cache-r5q9ees4\wheels\fc\53\3b\4b91b0b2c129eebb78724600d07d3f76527c4eb17bf2a5d929
Succ

In [3]:
p = Path(".")
root_folder = p.cwd().parent / 'pjm_run'
git_data_folder = root_folder  / 'git_data'
gen_data_folder = git_data_folder / 'gen_forecasts' 
scripts_foler = root_folder/ 'scripts'
outside_data_folder = root_folder / 'outside_data'
gen_data_folder = git_data_folder / 'gen_forecasts'
historical_load_data = git_data_folder / 'historical_load'
scratch_folder = root_folder  / 'scratch'

In [4]:

historical_load_data

WindowsPath('c:/Users/shubg/Desktop/GitHub/assetraMP/examples/pjm_run/git_data/historical_load')

### Demand Data

In [5]:
#Function to load demand data
def load_pjm_cleaned_hourly_demand(
        pjm_demand_file: Path,
        start_hour: datetime,
        end_hour: datetime) -> xr.DataArray:
    """Return hourly demand data as formatted data array.
    To use this function, download cleaned demand data from:

    https://github.com/truggles/EIA_Cleaned_Hourly_Electricity_Demand_Data

    Args:
        eia_930_cleaned_demand_file (Path): Path to hourly demand file
        start_hour (datetime): First timestamp to include
        end_hour (datetime): Last timestamp to include (inclusive)

    Returns:
        xr.DataArray: Hourly demand array with time dimension and datetime coordinates.
    """
    # read demand file
    pjm_demand_df = pd.read_csv(
        pjm_demand_file,
        usecols=["datetime_beginning_ept", "mw"],
        index_col="datetime_beginning_ept",
        parse_dates=True,
    )

    # remove leap day
    #pjm_demand_df = pjm_demand_df[~((pjm_demand_df.index.month == 2) & (pjm_demand_df.index.day == 29))]


    # keep cleaned demand demand
    pjm_hourly_demand_pd = pjm_demand_df["mw"].loc[start_hour:end_hour]

    # convert to xr.DataArray
    pjm_hourly_demand = xr.DataArray(
        data=pjm_hourly_demand_pd.values,
        coords=dict(
            time=pjm_hourly_demand_pd.index.values
        )
    )
    return pjm_hourly_demand

In [6]:
#Load demand data
pjm_cleaned_demand_file = Path( historical_load_data /  "pjm_load_2020.csv")
hourly_demand = load_pjm_cleaned_hourly_demand(
	pjm_cleaned_demand_file,
	start_hour="2020-01-01 00:00:00",
	end_hour="2020-12-31 23:00:00"
)

In [7]:
from assetra.system import EnergySystem
from assetra.system import EnergySystemBuilder

builder = EnergySystemBuilder()
unit_count = 0


In [8]:
# create demand unit
from assetra.units import DemandUnit

builder.add_unit(
    DemandUnit(
        id=unit_count,
        hourly_demand=hourly_demand
    )
)
unit_count += 1

### EIA 860 Data

In [9]:
#Test for just one year
thermal_generators = pd.read_csv(gen_data_folder / "thermal_2025.csv") #, index_col=0)
wind_generators = pd.read_csv(gen_data_folder / "wind_2025.csv")#, index_col=0)
solar_generators = pd.read_csv(gen_data_folder / "solar_2025.csv")#, index_col=0)
storage_generators = pd.read_csv(gen_data_folder / "batteries_2025.csv")#, index_col=0)
thermal_generators.reset_index(inplace=True)
storage_generators.reset_index(inplace=True)
solar_generators.reset_index(inplace=True)
wind_generators.reset_index(inplace=True)
thermal_generators.loc[(thermal_generators['MC'].isnull()), "MC"] = 9999


### Import MERRA

In [10]:
pow_gen_file_2020 = Path(outside_data_folder / "power_gen_data" / "pjm_power_generation_2020.nc")
pow_gen_dataset_2020 = xr.open_dataset(pow_gen_file_2020)


In [11]:

pow_gen_dataset = pow_gen_dataset_2020

def get_nearest_hourly_profile(
    latitude: float,
    longitude: float,
    array: xr.DataArray
) -> xr.DataArray:
    """Return time series corresponding to the nearest coordinate in a
    MERRA power generation data array.

    Args:
        latitude (float): Latitude relative to equator in degrees
        start_hour (datetime): Longitude relative to meridian in degrees
        array (xr.DataArray): "solar_capacity_factor", "wind_capacity_factor",
            or "temperature"

    Returns:
        xr.DataArray: Array with time dimension and datetime coordinates.
    """
    return array.sel(
            lat=latitude, 
            lon=longitude, 
            method="nearest"
        ).squeeze(drop=True)

def get_merra_power_generation_solar_cf(
    latitude: float,
    longitude: float) -> xr.DataArray:
    return get_nearest_hourly_profile(latitude, longitude, pow_gen_dataset["solar_capacity_factor"])

def get_merra_power_generation_wind_cf(
    latitude: float,
    longitude: float) -> xr.DataArray:
    return get_nearest_hourly_profile(latitude, longitude, pow_gen_dataset["wind_capacity_factor"])

def get_merra_power_generation_temperature(
    latitude: float,
    longitude: float) -> xr.DataArray:
    return get_nearest_hourly_profile(latitude, longitude, pow_gen_dataset["temperature"])

In [12]:
import pandas as pd

# load temperature dependent outage rate (tdfor) table
tdfor_table_file = Path( git_data_folder / "temperature_dependent_outage_rates.csv")
tdfor_table = pd.read_csv(tdfor_table_file, index_col=0)
tdfor_table = tdfor_table / 100 # percentages stored as integers

# create mapping table for tdfor table
tech_categories = {
    "CC" : ["Natural Gas Fired Combined Cycle"],
    "CT" : ["Natural Gas Fired Combustion Turbine","Landfill Gas"],
    "DS" : ["Natural Gas Internal Combustion Engine"],
    "ST" : ["Conventional Steam Coal","Natural Gas Steam Turbine"],
    "NU" : ["Nuclear"],
    "HD" : ["Conventional Hydroelectric","Solar Thermal without Energy Storage",
                   "Hydroelectric Pumped Storage","Solar Thermal with Energy Storage","Wood/Wood Waste Biomass"]
}

# create mapping from technology to category
tech_mapping = {tech : cat for cat, techs in tech_categories.items() for tech in techs}

def get_hourly_forced_outage_rate(hourly_temperature: xr.DataArray, technology: str) -> xr.DataArray:
    # index tdfor table by tech
    tdfor_map = tdfor_table[tech_mapping.get(technology, "Other")]
    map_temp_to_for = lambda hourly_temperature: tdfor_map.iloc[
            tdfor_map.index.get_indexer(hourly_temperature, method="nearest")
        ]
    return xr.apply_ufunc(
        map_temp_to_for,
        hourly_temperature
    ).rename("hourly_forced_outage_rate")


### Build Units

In [13]:
from assetra.units import StochasticUnit


for _, generator in thermal_generators.iterrows():
    # get hourly temperature
    hourly_temperature = get_merra_power_generation_temperature(
        generator["LAT"],
        generator["LON"]
    )

    # map temperature to hourly forced outage rate
    hourly_forced_outage_rate = get_hourly_forced_outage_rate(hourly_temperature, generator["Technology"])

    # get hourly capacity
    hourly_capacity = ( 
        xr.ones_like(hourly_temperature).rename("hourly_capacity") 
        * generator["Nameplate Capacity (MW)"]
    )

    # create assetra energy unit
    thermal_unit = StochasticUnit(
            id=unit_count,
            nameplate_capacity=generator["Nameplate Capacity (MW)"],
            hourly_capacity=hourly_capacity,
            hourly_forced_outage_rate=hourly_forced_outage_rate,
            marginal_cost = generator["MC"],
        )
    unit_count += 1
    
    # add unit to energy system
    builder.add_unit(thermal_unit)



In [14]:

# add solar 
for _, generator in solar_generators.iterrows():
    # get hourly temperature
    hourly_temperature = get_merra_power_generation_temperature(
        generator["LAT"],
        generator["LON"]
    )
    # get hourly temperature
    hourly_capacity = get_merra_power_generation_solar_cf(
        generator["LAT"],
        generator["LON"]
    ) * generator["Nameplate Capacity (MW)"]

    # map temperature to hourly forced outage rate
    hourly_forced_outage_rate = get_hourly_forced_outage_rate(hourly_temperature, generator["Technology"])

    # create assetra energy unit
    solar_unit = StochasticUnit(
            id=unit_count,
            nameplate_capacity=generator["Nameplate Capacity (MW)"],
            hourly_capacity=hourly_capacity,
            hourly_forced_outage_rate=hourly_forced_outage_rate,
            marginal_cost = generator["MC"], #0,
            )
    unit_count += 1
    
    # add unit to energy system
    builder.add_unit(solar_unit)

# add wind
for _, generator in wind_generators.iterrows():
    # get hourly temperature
    hourly_temperature = get_merra_power_generation_temperature(
        
        generator["LAT"],
        generator["LON"]
    )
    # get hourly temperature
    hourly_capacity = get_merra_power_generation_wind_cf(
        generator["LAT"],
        generator["LON"]
    ) * generator["Nameplate Capacity (MW)"]

    # map temperature to hourly forced outage rate
    hourly_forced_outage_rate = get_hourly_forced_outage_rate(hourly_temperature, generator["Technology"])


    # create assetra energy unit
    wind_unit = StochasticUnit(
            id=unit_count,
            nameplate_capacity=generator["Nameplate Capacity (MW)"],
            hourly_capacity=hourly_capacity,
            hourly_forced_outage_rate=hourly_forced_outage_rate,
            marginal_cost = generator["MC"],# 0,
            )
    unit_count += 1
    
    # add unit to energy system
    builder.add_unit(wind_unit)



In [15]:
from assetra.units import StorageUnit

#class_to_test = '8'
#class_to_test_lst = []

STORAGE_EFFICIENCY = 0.85

for _, generator in storage_generators.iterrows():
    storage_duration = generator["Nameplate Energy Capacity (MWh)"] / generator["Nameplate Capacity (MW)"]
    if storage_duration < 4:
        storage_class = '4'
    elif 4 <= storage_duration < 6:
        storage_class = '4'
    elif 6 <= storage_duration < 8:
        storage_class = '8'
    elif 8 <= storage_duration < 10:
        storage_class = '8'
    else:
        storage_class = '4'

    #if storage_class == class_to_test:
     #   class_to_test_lst.append(generator)
    #else:
    storage_unit = StorageUnit(
        id=unit_count,
        nameplate_capacity=generator["Nameplate Capacity (MW)"],
        charge_rate=generator["Nameplate Capacity (MW)"],
        discharge_rate=generator["Nameplate Capacity (MW)"],
        charge_capacity=generator["Nameplate Energy Capacity (MWh)"],
        roundtrip_efficiency = STORAGE_EFFICIENCY,
        storage_duration = generator["Nameplate Energy Capacity (MWh)"] / generator["Nameplate Capacity (MW)"],
        storage_class = storage_class       
    )
    unit_count += 1

        # add unit to energy system
    builder.add_unit(storage_unit)
    #class_to_test_df = pd.DataFrame(class_to_test_lst, columns=storage_generators.columns)

### Build and Run the Model

In [16]:
from assetra.system import EnergySystem
from assetra.system import EnergySystemBuilder

In [17]:
system_dir = Path(scratch_folder / "pjm_energy_system_TEST")

energy_system = builder.build()
energy_system.save(system_dir)


In [18]:
energy_system = EnergySystem()
energy_system.load(system_dir)

In [19]:
print("# of Units:", energy_system.size)
print("Sys. Capacity (MW):", round(energy_system.system_capacity))

# of Units: 4039
Sys. Capacity (MW): 216459


In [20]:
from assetra.simulation import ProbabilisticSimulation

simulation = ProbabilisticSimulation(
    start_hour="2020-01-01 00:00:00",
    end_hour="2020-12-31 23:00:00",
    trial_size=1
)

simulation.assign_energy_system(energy_system)
simulation.run()

The ProbabilisticSimulation object generates a net hourly capacity matrix, representing net system capacity in each Monte Carlo trial. We can access a copy of this matrix to analyze shortfalls.

In [21]:
# convert net hourly capacity matrix to pandas dataframe with risk hours only
shortfall_matrix_pd = simulation.net_hourly_capacity_matrix.where(lambda c: c < 0).to_pandas().T.dropna(how="all")
shortfall_matrix_pd

trial,0
time,
2020-01-03 17:00:00,-3189.544124
2020-01-03 18:00:00,-9244.638282
2020-01-03 19:00:00,-1868.160165
2020-01-05 08:00:00,-669.506201
2020-01-06 07:00:00,-12048.201775
...,...
2020-12-30 21:00:00,-13107.425634
2020-12-31 17:00:00,-12185.887334
2020-12-31 18:00:00,-13348.654417


### Calculating Reliability Metrics


We can calculate hourly loss of load probability from the net hourly capacity matrix

In [22]:
# get loss of load probability
loss_of_load_prob = shortfall_matrix_pd.count(axis=1) / shortfall_matrix_pd.shape[1]

# show top 10 risk hours
loss_of_load_prob.sort_values(ascending=False)[:10]

time
2020-01-03 17:00:00    1.0
2020-08-21 14:00:00    1.0
2020-08-22 18:00:00    1.0
2020-08-22 19:00:00    1.0
2020-08-22 20:00:00    1.0
2020-08-22 21:00:00    1.0
2020-08-22 22:00:00    1.0
2020-08-22 23:00:00    1.0
2020-08-23 08:00:00    1.0
2020-08-23 09:00:00    1.0
dtype: float64

In [23]:
from assetra.metrics import ExpectedUnservedEnergy

# instantiate eue model
eue_model = ExpectedUnservedEnergy(simulation)
eue = eue_model.evaluate()

print("System EUE:", round(eue, 2), "MWh")

System EUE: 139823907.68 MWh


In [24]:
from assetra.metrics import LossOfLoadHours, LossOfLoadDays, LossOfLoadFrequency
import pandas as pd

adequacy = pd.Series(dtype=float)

for name, metric in [
    ("EUE (MWh)", ExpectedUnservedEnergy),
    ("LOLH (h)", LossOfLoadHours),
    ("LOLD (d)", LossOfLoadDays),
    ("LOLF (#)", LossOfLoadFrequency)
]:
    adequacy[name] = metric(simulation).evaluate()

# show results
adequacy.round(1)

EUE (MWh)    139823907.7
LOLH (h)          2844.0
LOLD (d)           234.0
LOLF (#)           267.0
dtype: float64

In [25]:
adequacy["Average Outage Duration (h)"] = adequacy["LOLH (h)"] / adequacy["LOLF (#)"]
adequacy["Average Shortfall (MW)"] = adequacy["EUE (MWh)"] / adequacy["LOLH (h)"]

# show results
adequacy.round(1)


EUE (MWh)                      139823907.7
LOLH (h)                            2844.0
LOLD (d)                             234.0
LOLF (#)                             267.0
Average Outage Duration (h)           10.7
Average Shortfall (MW)             49164.5
dtype: float64

### Calculating UCAP

In [ ]:
#Extracting unit level charge/discharge in each hour for this simulation
#storage_performance = simulation.unit_performance["StorageUnit"]
#Extracting the performance of one storage asset and converting to a dataframe
#specific_performance = storage_performance.sel(energy_unit = 1)
#specific_performance.to_dataframe(name = 'performance').reset_index()
#plot the dataframe for just january
#specific_performance = specific_performance.sel(time=slice("2020-01-01", "2020-01-31"))
#import matplotlib.pyplot as plt
#fig, ax = plt.subplots(figsize=(10, 6))
#specific_performance.to_dataframe(name = 'performance').reset_index().plot(x='time', y='performance', ax=ax)

In [121]:
storage_performance = simulation.unit_performance["StorageUnit"]

In [123]:
#Converting this xarray into a dataframe

# Convert 'performance' to a DataFrame
df_performance = performance.to_dataframe(name='performance').reset_index()

# Convert 'nameplate_capacity' to a DataFrame
df_nameplate_capacity = nameplate_capacity.to_dataframe(name='nameplate_capacity').reset_index()

# Convert 'charge_capacity' to a DataFrame
df_charge_capacity = nameplate_energy_capacity.to_dataframe(name='nameplate_energy_capacity').reset_index()

# Merge the nameplate capacity and charge capacity DataFrames on 'energy_unit'
df_combined = pd.merge(df_nameplate_capacity, df_charge_capacity, on='energy_unit', how='inner')

# Concatenate df_combined and df_performance along axis 1 (columns)
batteries_performance_df = pd.concat([df_combined, df_performance], axis=1)

#renaming the first energy_unit to battery_unit_num
batteries_performance_df.columns.values[0] = 'battery_unit_num'

In [122]:
#Calculating the annual energy capacity of each battery

#Identifying the max duration of each battery
batteries_performance_df['duration'] = batteries_performance_df['nameplate_energy_capacity']/batteries_performance_df['nameplate_capacity']
#classiying them as 4 or 8 hour batteries
batteries_performance_df['class'] = batteries_performance_df['duration'].apply(lambda x: '4' if x <6 else '8')
#calculating the total annual energy capacity of each battery
num_hours = performance.sizes["time"] #number of hours in this year
batteries_performance_df['annual_energy_cap_mwh'] = (num_hours/batteries_performance_df['duration'])*batteries_performance_df['nameplate_energy_capacity']

RPA = Au/Ac ie the ratio of the availability of a unit to that of its whole class.

Au = summation(discharges_over_time_mwh)/nameplate_energy_capacity_mwh

Ac =  summation(all Au_numerators)/summation(all Au_denominators)

In [120]:
#Calculating RPA
Au_numerator = batteries_performance_df[batteries_performance_df['performance'] > 0].groupby('energy_unit')['performance'].sum()
Au_denominator = batteries_performance_df.loc[0:58,'annual_energy_cap_mwh']
Au = Au_numerator/Au_denominator

Ac_numerator = Au_numerator.sum()
Ac_denominator = Au_denominator.sum()
Ac = Ac_numerator/Ac_denominator

RPAu = Au/Ac
RPAu

energy_unit
0      0.933658
1     16.709461
2      9.564981
3     19.110642
4      1.836156
5      0.954801
6      1.193501
7      0.954801
8     23.870025
9      1.734747
10     4.238542
11     0.605506
12     1.905758
13     0.476271
14     0.604693
15     0.960159
16     3.167615
17    18.997924
18     0.958743
19     0.958743
20    18.983105
21    18.983105
22     1.054145
23     1.896052
24     0.948026
25    31.569381
26     1.803613
27     1.578162
28     0.946897
29     0.127522
30     1.572814
31     1.784477
32     0.079643
33     0.095571
34     1.992033
35     0.041860
36     0.435349
37     0.669767
38     0.041860
39     0.828837
40     0.828837
41     0.041860
42     0.418605
43     0.501931
44     0.834300
45     0.292005
46     0.258633
47     0.627203
48     0.505271
49     0.041656
50     0.208279
51     0.345742
52     0.170788
53     0.033325
54     0.520696
55     0.520696
56     0.324915
57     0.083311
58     0.041656
dtype: float64

### Not Using this ELCC method - do not run
Quantify resource contribution (ELCC)

Resource contribution is a typical extension of resource adequacy analysis. The assetra package implements effective load-carrying capability (ELCC) to quantify resource contribution. When we instantiate an EffectiveLoadCarryingCapability object, the base system will automatically be evaluated according to the ResourceAdequacyMetric type we provide. In the following example, we indicate that resource adequacy should be defined as EUE (e.g. rather than LOLH) by passing ExpectedUnservedEnergy (the class not an instance) as the last parameter to the ELCC instance.


In [ ]:
builder = EnergySystemBuilder()

for _, generator in class_to_test_df.iterrows():
    storage_duration = generator["Nameplate Energy Capacity (MWh)"] / generator["Nameplate Capacity (MW)"]
    if storage_duration < 4:
        storage_class = '4'
    elif 4 <= storage_duration < 6:
        storage_class = '4'
    elif 6 <= storage_duration < 8:
        storage_class = '8'
    elif 8 <= storage_duration < 10:
        storage_class = '8'
    else:
        storage_class = '8'
        
    storage_unit = StorageUnit(
        id=unit_count,
        nameplate_capacity=generator["Nameplate Capacity (MW)"],
        charge_rate=generator["Nameplate Capacity (MW)"],
        discharge_rate=generator["Nameplate Capacity (MW)"],
        charge_capacity=generator["Nameplate Energy Capacity (MWh)"],
        roundtrip_efficiency=STORAGE_EFFICIENCY,
        storage_duration = generator["Nameplate Energy Capacity (MWh)"] / generator["Nameplate Capacity (MW)"],
        storage_class = storage_class   
    )
    unit_count += 1
    builder.add_unit(storage_unit)

    # add unit to energy system
additional_system = builder.build()

In [ ]:
additional_system.system_capacity

355.29999999999995

In [ ]:
from assetra.contribution import EffectiveLoadCarryingCapability
from assetra.simulation import ProbabilisticSimulation
from assetra.metrics import ExpectedUnservedEnergy

# initialize elcc model
elcc_model = EffectiveLoadCarryingCapability(
    energy_system,
    ProbabilisticSimulation(
        start_hour="2020-01-01 00:00:00",
        end_hour="2020-12-31 23:00:00",
        trial_size=10
    ),
    ExpectedUnservedEnergy
)


In [ ]:
additional_system = builder.build()

In [ ]:
elcc = elcc_model.evaluate(additional_system)
elcc_pct = elcc / additional_system.system_capacity * 100
elcc_pct

14.453125